In [154]:
import requests
import json
import xmltodict
import urllib.parse
import opencitingpy
import datetime
import sqlite3
import os

# Setup sqlite Datenbank
Datenbank wird im Verzeichnis "C:\MA_Pethke_3992454" abgelegt

In [155]:
# Anlegen von Datenbankspeicherort, falls er nicht existiert
path = "C:\MA_Pethke_3992454"
if not os.path.exists(path):
    os.makedirs(path)

# Verbindung zu DB aufbauen, DB wird automatisch erstellt, wenn keine da ist
con = sqlite3.connect("C:\MA_Pethke_3992454\MA_3992454.db")
cur = con.cursor()

In [177]:
## Tabellen Schema erstellen
#------------------- Tabelle Dokument --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Dokument(PMID INTEGER, titel TEXT, pubdate DATE)")

#------------------- Tabelle Zitationen --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Zitationen(PMID INTEGER, jahr YEAR, citecount INTEGER)")

#------------------- Tabelle Mesh --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Mesh(UI VARCHAR, name TEXT, datum DATE)")

#------------------- Tabelle Abfrage --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Abfrage(query TEXT, PMID INTEGER, ranking INTEGER, mesh VARCHAR)")
con.commit()

# PubMed
## Teil 1
### Query mit Stichworten --> Liste der relevanten Dokumente
queryPubMed: Stichwortartige Suche <br>
idList: Liste mit PMIDs <br>
countResult: Anzahl (maximal) zurückgegebener Dokumente <br>

### URL 
db: welche Datenbank wird durchsucht (pubmed) <br>
term: queryPubMed <br>
RetMax: countResult <br>
sort: Art der Sortierung (default: relevance) <br>
retmod: Format der zurückgelieferten Daten <br>



In [157]:
## Liste der "relevanten" Dokumente
queryPubMed = "head injury"
countResult = 10
responsePubMedID = requests.get('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term='+urllib.parse.quote(queryPubMed)+'&RetMax='+urllib.parse.quote(str(countResult))+'&sort=relevance&retmode=json')
responsePubMedID_json = json.loads(responsePubMedID.text)
idList = responsePubMedID_json["esearchresult"]["idlist"]
print(idList)

['25335757', '36637470', '22942365', '9279162', '9425562', '9339463', '3330962', '3931543', '27509653', '23431655']


## Teil 2
### Query mit PMID aus Teil 1 --> Liste der MeSH zu dem jeweiligen Dokument
Für jedes Dokument (Identifier: PMID) wird eine Detailsuche durchgeführt. <br>
Extrahierte Infos: <br>
Titel <br>
Liste der MeSH: [[PMID, [MeSH UI, MeSH Name], [MeSH UI, MeSH Name]] ,[PMID, [MeSH UI, MeSH Name], [MeSH UI, MeSH Name]] ,  ...] (beliebig viele MeSH pro Dokument)<br>
Datum Veröffentlichung: Wann wurde das Dokument veröffentlicht <br>

In [158]:
## Liste der MeSHs (zu allen Dokumenten)
ranking = 0
for pmid in idList:
    ranking = ranking + 1
    # print(id)
    # temporäre Liste zum Anhängen an meshList
    # Abfrage zu einer einzelnen PMID
    response_PubMedMesh = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id="+urllib.parse.quote(str(pmid)))
    # Formatierung
    xpars = xmltodict.parse(response_PubMedMesh.text)
    json_str = json.dumps(xpars, indent=4)
    json_data = json.loads(json_str)
    title = json_data["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"]["Article"]["ArticleTitle"]
    # Nicht alle Dokumente haben ein eigenes Veröffentlichungsdatum, dann wird sich auf das DateCompleted von PubMed berufen (TODO: recherchieren, was das genau ist)
    try:
        pubDateAll = json_data["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"]["Article"]["ArticleDate"]
        pubDate = datetime.datetime(int(pubDateAll["Year"]), int(pubDateAll["Month"]), int(pubDateAll["Day"]))
    except:
        pubDateAll = json_data["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"]["DateCompleted"]
        pubDate = datetime.datetime(int(pubDateAll["Year"]), int(pubDateAll["Month"]), int(pubDateAll["Day"]))
    cur.execute("INSERT INTO Dokument (PMID, titel, pubdate) VALUES ('"+str(pmid)+"', '"+str(title)+"', '"+str(pubDate)+"')")
    con.commit()

    meshAll = json_data["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"]["MeshHeadingList"]["MeshHeading"]
    for mesh in meshAll:
        meshUI = mesh["DescriptorName"]["@UI"]
        cur.execute("INSERT INTO Abfrage (query, PMID, ranking, mesh) VALUES ('"+str(queryPubMed)+"', '"+str(pmid)+"', '"+str(ranking)+"', '"+str(meshUI)+"')")
        cur.execute("SELECT * FROM Mesh WHERE UI = '"+str(meshUI)+"'")
        # Mesh wird folgend initial gelistet, wenn noch nicht geschehen
        if cur.fetchall() == []:
            meshName = mesh["DescriptorName"]["#text"]
            cur.execute("INSERT INTO Mesh (UI, name) VALUES ('"+str(meshUI)+"', '"+str(meshName)+"')")
        con.commit()
        

# MeSH
## Wann wurde das MeSH hinzugefügt?
Dies ist für jedes MeSH in allen Dokumenten zu ermitteln

In [159]:
cur.execute("SELECT DISTINCT mesh FROM Abfrage WHERE query = '"+str(queryPubMed)+"'")
meshPerQuery = [x[0] for x in cur.fetchall()]
print(meshPerQuery)

['D000293', 'D000328', 'D002648', 'D006259', 'D006801', 'D017410', 'D012307', 'D013124', 'D014057', 'D014218', 'D007223', 'D002675', 'D002649', 'D012640', 'D059906', 'D061906', 'D058109', 'D004677', 'D015600', 'D010102', 'D000059', 'D004630', 'D015994', 'D010988', 'D012606', 'D000818', 'D016489', 'D013313', 'D009733', 'D010371', 'D001930', 'D002490', 'D019586', 'D014481', 'D004244', 'D004780', 'D004834', 'D006311', 'D006850', 'D008297', 'D001523', 'D009043', 'D009055', 'D009422', 'D014786', 'D019468', 'D005260', 'D011247', 'D011248', 'D049188', 'D004632', 'D009730', 'D012887']


In [160]:
for mesh in meshPerQuery:
    response = requests.get('https://id.nlm.nih.gov/mesh/'+str(mesh)+'.json', headers={'Accept': 'application/json'})
    data = response.json()
    cur.execute("UPDATE Mesh SET datum = '"+data["dateCreated"]+"' WHERE UI = '"+str(mesh)+"'")
    con.commit()

# Semantic Scholar
Zitationen pro Jahr

In [161]:
def getCitations(citations, citationsPerYearLoc):
    for cite in citations:
        year = str(cite["citingPaper"]["year"])
        if year not in citationsPerYearLoc:
            citationsPerYearLoc[year] = 1
        else:
            citationsPerYearLoc[year] = citationsPerYearLoc[year] + 1
    return citationsPerYearLoc

In [168]:
def getCitationCount(offset, pmid):
    response = requests.get("https://api.semanticscholar.org/graph/v1/paper/PMID:"+str(pmid)+"/citations?fields=year&offset="+str(offset)+"&limit="+str(1000))
    json_str = json.loads(response.text)
    # print(json_str)
    citations = json_str["data"]
    citationCount = len(citations)
    return citations, citationCount

In [181]:
for pmid in idList:
    citationsPerYear = {}
    offset = 0
    citationsTemp = getCitationCount(offset, pmid)
    while citationsTemp[1] != 0:
        getCitations(citationsTemp[0], citationsPerYear)
        for year, citationcount in citationsPerYear.items():
            cur.execute("INSERT INTO Zitationen (PMID, jahr, citecount) VALUES ('"+str(pmid)+"', '"+str(year)+"', '"+str(citationcount)+"')")
            con.commit()
        offset = offset + 1000
        citationsTemp = getCitationCount(offset, pmid)

